# Data Cleaning

AIS data

In [1]:
import pandas as pd
import numpy as np
import os
import re

All the AIS files are the same time period: 01JUL2024 - 01JUL2025
AIS data is from Global Fishing Watch and utilizes AIS data and their algorithm to identify 'apparent fishing efforts'. This is aimed to be just fishing vessel movement and activity information.

Data is broken down by EEZ and High Seas Pockets (for areas outside the EEZs) and concentrated in the highy traversed parts of the WCPFC's Convention Area.

In [109]:
directory_path = '../data/AIS_data'

merged_df = pd.DataFrame(columns=['flag', 'vessel_name', 'mmsi', 'imo'])
longer_df = pd.DataFrame(columns=['flag', 'vessel_name', 'mmsi', 'imo'])


for filename in os.listdir(directory_path):
    file_path = os.path.join(directory_path, filename)
    print(file_path)
    if file_path != "../data/AIS_data/.DS_Store":
        df = pd.read_csv(file_path)

        df_filter = df[['Flag', 'Vessel Name', 'MMSI', 'IMO', 'Apparent Fishing Hours']].copy()

        area_name = file_path[17:][:-4]
        print(area_name)

        df_filter.columns = ['flag', 'vessel_name', 'mmsi', 'imo',area_name]

        df_filter['mmsi'] = df_filter['mmsi'].apply(lambda x: str(int(x)) if pd.notna(x) else x)
        df_filter['imo'] = df_filter['imo'].apply(lambda x: str(int(x)) if pd.notna(x) else x)

        df_filter['vessel_name'] = df_filter['vessel_name'].astype(str).str.upper()
        df_filter['vessel_name'] = df_filter['vessel_name'].str.replace("F/V", "", regex=False).str.replace("FV ", "", regex=False).str.replace("\"", "", regex=False).str.replace(".", "", regex=False).str.replace('"', '', regex=False).str.strip()

        df_filter = df_filter.drop_duplicates()

        merged_df = pd.merge(merged_df, df_filter, on = ['flag', 'vessel_name', 'mmsi', 'imo'], how = 'outer')

        # make a longer dataframe for network creation
        df_filter = df[['Flag', 'Vessel Name', 'MMSI', 'IMO', 'Apparent Fishing Hours']].copy()
        df_filter['fishing_area'] = area_name
        df_filter.columns = ['flag', 'vessel_name', 'mmsi', 'imo','fishing_hours', 'fishing_area']
        df_filter = df_filter.drop_duplicates()
        df_filter = df_filter.replace(['', 'nan'], pd.NA)

        longer_df = pd.concat([longer_df, df_filter], ignore_index=True)

# Replace blanks and string 'nan' with actual NaN
merged_df = merged_df.replace(['', 'nan'], pd.NA)
# resolve duplicate vessels
# group by the first four columns and compute the mean for the rest
grouped_df = merged_df.groupby(['flag', 'vessel_name', 'mmsi', 'imo'], dropna=False).mean(numeric_only=True).reset_index()

# check columns
print(grouped_df.columns)
print(longer_df.columns)

# sort dataframes
longer_df = longer_df.sort_values(by = ['flag','vessel_name'], ascending=True).copy()
grouped_df = grouped_df.sort_values(by = ['flag','vessel_name'], ascending=True).copy()

# write to a csv
grouped_df.to_csv("../data/merged_ais_data.csv", index=False)
longer_df.to_csv('../data/longer_ais_data.csv', index=False)

../data/AIS_data/rmi_eez.csv
rmi_eez
../data/AIS_data/tonga_eez.csv
tonga_eez
../data/AIS_data/.DS_Store
../data/AIS_data/nauru_eez.csv
nauru_eez
../data/AIS_data/vanuatu_eez.csv
vanuatu_eez
../data/AIS_data/hsp2.csv
hsp2
../data/AIS_data/hsp3.csv
hsp3
../data/AIS_data/hsp1.csv
hsp1
../data/AIS_data/usa2_eez.csv
usa2_eez
../data/AIS_data/hsp4.csv
hsp4
../data/AIS_data/hsp5.csv
hsp5
../data/AIS_data/usa_eez.csv
usa_eez
../data/AIS_data/hsp7.csv
hsp7
../data/AIS_data/hsp6.csv
hsp6
../data/AIS_data/palau_eez.csv
palau_eez
../data/AIS_data/fsm_eez.csv
fsm_eez
../data/AIS_data/kirbati2_eez.csv
kirbati2_eez
../data/AIS_data/png_eez.csv
png_eez
../data/AIS_data/fiji_eez.csv
fiji_eez
../data/AIS_data/tuvalu_eez.csv
tuvalu_eez
../data/AIS_data/kiribati1_eez.csv
kiribati1_eez
../data/AIS_data/solomon_islands_eez.csv
solomon_islands_eez
Index(['flag', 'vessel_name', 'mmsi', 'imo', 'rmi_eez', 'tonga_eez',
       'nauru_eez', 'vanuatu_eez', 'hsp2', 'hsp3', 'hsp1', 'usa2_eez', 'hsp4',
       'hsp5',

/var/folders/9r/1lhfzshj5bvf1d4ybbng0kr00000gn/T/ipykernel_11002/1382724485.py:37: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  longer_df = pd.concat([longer_df, df_filter], ignore_index=True)


# Cleaning the Registered Fishing Vessel List (RFV)

In [45]:
# to convert to two letter country code to three
import pycountry

# convert 2-letter country code to 3-letter code
def alpha2_to_alpha3(code):
    try:
        return pycountry.countries.get(alpha_2=code).alpha_3
    except:
        return None 
    
# convert country name to 3-letter code
def name_to_alpha3(name):
    try:
        return pycountry.countries.get(name=name).alpha_3
    except:
        return None 

In [39]:
# vessel type csv file
vessel_tyes = pd.read_csv('../data/standard_code_lists/Fishery_Vessel_Types.csv')

# function to convert vessel type
def get_vessel_type_from_abbr(abbr_code):
    match = vessel_tyes[vessel_tyes["Standard Abbreviation Code"] == abbr_code]
    if not match.empty:
        return match.iloc[0]["Vessel Type"]
    else:
        return "Unknown abbreviation"

In [57]:
# load file
rfv = pd.read_csv('../data/WCPFC_raw_RFV.csv')

# get only the columns we need
good_cols = ["Name of fishing vessel","Flag of fishing vessel","Port of registry","Name of the owner or owners","Address of the owner or owners","Type of vessel"]
rfv_sub = rfv[good_cols]
# rename columns
rfv_sub.columns = ["vessel_name", "flag_2", "home_port",'owner','ownder_address','vessel_type']

# convert two letter code to three letter country code
rfv_sub['flag'] = rfv_sub['flag_2'].apply(alpha2_to_alpha3)

# convert fishing vessel type
rfv_sub['vessel_type'] = rfv_sub['vessel_type'].apply(get_vessel_type_from_abbr)

print(rfv_sub.head())

       vessel_name flag_2 home_port  \
0       OCEAN STAR     PA    PA-PTY   
1        MONTELAPE     SV    SV-LUN   
2       MONTELUCIA     SV    SV-LUN   
3       MONTEROCIO     SV    SV-LUN   
4  CHANG YI NO.368     TW    TW-KHH   

                                            owner  \
0                          MED BFT VESSEL LIMITED   
1  Oakcity Tuna Fishing Corporation, S.A. de C.V.   
2    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
3    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
4                   HAI CHAN YANG FISHERY CO.,LTD   

                                      ownder_address        vessel_type flag  
0  Hangar, Triq it-Trunciera, Marsaxlokk, MXK 152...       Fish carrier  PAN  
1  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV  
2  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV  
3  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV  
4  No. 971-6, Wufang Rd., Xinyuan Township, Pingt..

/var/folders/9r/1lhfzshj5bvf1d4ybbng0kr00000gn/T/ipykernel_17551/3083843949.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfv_sub['flag'] = rfv_sub['flag_2'].apply(alpha2_to_alpha3)
/var/folders/9r/1lhfzshj5bvf1d4ybbng0kr00000gn/T/ipykernel_17551/3083843949.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfv_sub['vessel_type'] = rfv_sub['vessel_type'].apply(get_vessel_type_from_abbr)


In [58]:
# parse address
def parse_state_country(address):
    address = str(address)
    parts = address.split(',')
    if len(parts) >= 2:
        state = parts[-2].strip()
        country = parts[-1].strip()
        # manually fix Taiwan
        if country == "Taiwan (R.O.C.)":
            country = "Taiwan"
            country_code = "TWN"
        # convert to country code
        country_code = name_to_alpha3(country)
        return state, country_code
    return None, None

rfv_sub['owner_location'] = rfv_sub['ownder_address'].apply(parse_state_country)

print(rfv_sub.head())

       vessel_name flag_2 home_port  \
0       OCEAN STAR     PA    PA-PTY   
1        MONTELAPE     SV    SV-LUN   
2       MONTELUCIA     SV    SV-LUN   
3       MONTEROCIO     SV    SV-LUN   
4  CHANG YI NO.368     TW    TW-KHH   

                                            owner  \
0                          MED BFT VESSEL LIMITED   
1  Oakcity Tuna Fishing Corporation, S.A. de C.V.   
2    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
3    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
4                   HAI CHAN YANG FISHERY CO.,LTD   

                                      ownder_address        vessel_type flag  \
0  Hangar, Triq it-Trunciera, Marsaxlokk, MXK 152...       Fish carrier  PAN   
1  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
2  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
3  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
4  No. 971-6, Wufang Rd., Xinyuan Township, Pi

/var/folders/9r/1lhfzshj5bvf1d4ybbng0kr00000gn/T/ipykernel_17551/3860550387.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfv_sub['owner_location'] = rfv_sub['ownder_address'].apply(parse_state_country)


In [63]:
# covert home port list
port_codes = pd.read_csv('../data/standard_code_lists/port_code_list.csv')

# function to convert vessel type
def get_port_code(abbr_code):
    match = port_codes[port_codes["Port Code"] == abbr_code]
    if not match.empty:
        port = match.iloc[0]["Port Name"]
        country = match.iloc[0]["Location Country"]            
        return port, country
    else:
        return None, None
    
rfv_sub['h_port'] = rfv_sub['home_port'].apply(get_port_code)

print(rfv_sub.head())

       vessel_name flag_2 home_port  \
0       OCEAN STAR     PA    PA-PTY   
1        MONTELAPE     SV    SV-LUN   
2       MONTELUCIA     SV    SV-LUN   
3       MONTEROCIO     SV    SV-LUN   
4  CHANG YI NO.368     TW    TW-KHH   

                                            owner  \
0                          MED BFT VESSEL LIMITED   
1  Oakcity Tuna Fishing Corporation, S.A. de C.V.   
2    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
3    OAKCITY TUNA FISHING CORPORATION S.A de C.V.   
4                   HAI CHAN YANG FISHERY CO.,LTD   

                                      ownder_address        vessel_type flag  \
0  Hangar, Triq it-Trunciera, Marsaxlokk, MXK 152...       Fish carrier  PAN   
1  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
2  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
3  CALLE LOMA LINDA, NO.251, COLONIA SAN BENITOSA...  Tuna purse seiner  SLV   
4  No. 971-6, Wufang Rd., Xinyuan Township, Pi

/var/folders/9r/1lhfzshj5bvf1d4ybbng0kr00000gn/T/ipykernel_17551/184873764.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  rfv_sub['h_port'] = rfv_sub['home_port'].apply(get_port_code)
